In [32]:
import subprocess
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import matplotlib.pyplot as plt
import os
import pickle
import numpy as np

In [40]:
def faToTwoBit(fasta_file: str):
    twobit_file = fasta_file.split(".")[0] + ".2bit"

    cmd = [
        "faToTwoBit",
        f"{fasta_file}",
        f"{twobit_file}"
    ]

    subprocess.run(cmd, check=True)
    print(f"Successfully created {twobit_file}")


In [2]:
def run_blat(reference_2bit, query_fasta, output_psl, tile_size=10, step_size=5):
    cmd = [
        "blat",
        f"-tileSize={tile_size}",
        f"-stepSize={step_size}",
        reference_2bit,
        query_fasta,
        output_psl
    ]
    print(f"Running BLAT: {' '.join(cmd)}")
    subprocess.run(cmd, check=True)

In [3]:
def parse_psl(psl_path):
    with open(psl_path) as f:
        lines = f.readlines()

    # Skip header lines
    data_lines = [line for line in lines if line.strip() and line[0].isdigit()]

    columns = [
        "matches", "misMatches", "repMatches", "nCount", "qNumInsert", "qBaseInsert",
        "tNumInsert", "tBaseInsert", "strand", "qName", "qSize", "qStart", "qEnd",
        "tName", "tSize", "tStart", "tEnd", "blockCount", "blockSizes",
        "qStarts", "tStarts"
    ]

    df = pd.DataFrame([line.strip().split('\t') for line in data_lines], columns=columns)
    df[["matches", "misMatches", "qSize", "qStart", "qEnd"]] = df[["matches", "misMatches", "qSize", "qStart", "qEnd"]].astype(int)

    df["aligned_length"] = df["qEnd"] - df["qStart"]
    df["percent_identity"] = df["matches"] / (df["matches"] + df["misMatches"] + 1e-6)
    df["coverage"] = df["aligned_length"] / (df["qSize"] + 1e-6)

    return df

In [4]:
def summarize_blat_results(df):
    top_hits = df.sort_values("percent_identity", ascending=False).drop_duplicates("qName")

    print("Top-hit identity statistics:")
    print(top_hits["percent_identity"].describe())

    num_exact_matches = ((top_hits["percent_identity"] == 1.0) & (top_hits["coverage"] == 1.0)).sum()
    print(f"\nPerfect duplicates (identity=1.0 and full coverage): {num_exact_matches} / {len(top_hits)}")

    return top_hits

In [6]:
def plot_alignment_stats(df, output_dir="plots"):
    os.makedirs(output_dir, exist_ok=True)

    top_hits = df.sort_values("percent_identity", ascending=False).drop_duplicates("qName")

    plt.figure(figsize=(8, 5))
    plt.hist(top_hits["percent_identity"], bins=50, color="steelblue", edgecolor="black")
    plt.xlabel("Percent Identity")
    plt.ylabel("Frequency")
    plt.title("Top Hit: Percent Identity")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "percent_identity_hist.png"))
    plt.close()

    plt.figure(figsize=(8, 5))
    plt.hist(top_hits["coverage"], bins=50, color="orange", edgecolor="black")
    plt.xlabel("Query Coverage")
    plt.ylabel("Frequency")
    plt.title("Top Hit: Coverage")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "coverage_hist.png"))
    plt.close()

    plt.figure(figsize=(6, 6))
    plt.scatter(top_hits["coverage"], top_hits["percent_identity"], alpha=0.5)
    plt.xlabel("Coverage")
    plt.ylabel("Percent Identity")
    plt.title("Identity vs. Coverage (Top Hits)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, "identity_vs_coverage.png"))
    plt.close()

In [7]:
def csv_to_fasta(csv_path, fasta_path):
    df = pd.read_csv(csv_path)
    sequences = df["val_seq"]

    records = []
    for i, seq in enumerate(sequences):
        seq_clean = str(seq).strip().upper()
        if not set(seq_clean).issubset(set("ACGT")):
            continue
        record = SeqRecord(Seq(seq_clean), id=f"valseq_{i}", description="")
        records.append(record)

    SeqIO.write(records, fasta_path, "fasta")
    print(f"Wrote {len(records)} sequences to {fasta_path}")

## Test Generated against Generated to See if BLAT works

In [ ]:
csv_to_fasta("../workdir/aptatrans_training_attempt2_2025-08-04_13-15-22/val_0.csv", "generated_aptamer_sequences.fasta")

Wrote 12254 sequences to generated_sequences.fasta


In [24]:
ref_2bit = "generated_aptamer_sequences.2bit"
query_fasta = "generated_aptamer_sequences.fasta"
out_psl = "blat_output.psl"

In [25]:
run_blat(ref_2bit, query_fasta, out_psl)

Running BLAT: blat -tileSize=10 -stepSize=5 generated_aptamer_sequences.2bit generated_aptamer_sequences.fasta blat_output.psl
Loaded 1568512 letters in 12254 sequences
Searched 1568512 bases in 12254 sequences


In [26]:
df = parse_psl(out_psl)

In [27]:
top_hits = summarize_blat_results(df)


Top-hit identity statistics:
count    1.225400e+04
mean     1.000000e+00
std      3.330805e-16
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      1.000000e+00
Name: percent_identity, dtype: float64

Perfect duplicates (identity=1.0 and full coverage): 0 / 12254


In [28]:
plot_alignment_stats(top_hits)

# Compare DeepFlyBrain to Aptamers

In [33]:
index_to_base = np.array(['A', 'C', 'G', 'T'])

def onehot_to_dna(onehot_array):
    # Get the index of the "1" in the last dimension
    indices = np.argmax(onehot_array, axis=-1)  # Shape: (104665, 500)
    # Map to letters
    sequences = ["".join(index_to_base[row]) for row in indices]
    return sequences

In [38]:
def save_to_fasta(sequences, output_path, header_prefix="seq"):
    with open(output_path, "w") as fasta_file:
        for i, seq in enumerate(sequences):
            fasta_file.write(f">{header_prefix}_{i}\n")
            fasta_file.write(seq + "\n")

In [16]:
flybrain_pik = pickle.load(open("../data/the_code/General/data/DeepFlyBrain_data.pkl", 'rb'))

In [19]:
flybrain_data = flybrain_pik['nonAugmented_data']

In [34]:
flybrain_data = onehot_to_dna(flybrain_data)

In [39]:
save_to_fasta(flybrain_data, "flybrain_natural_sequences.fasta")

In [41]:
ref_2bit = "flybrain_natural_sequences.2bit"
query_fasta = "generated_aptamer_sequences.fasta"
out_psl = "blat_flybrain_natural_vs_generated_aptamers_output.psl"

In [42]:
run_blat(ref_2bit, query_fasta, out_psl)

Running BLAT: blat -tileSize=10 -stepSize=5 flybrain_natural_sequences.2bit generated_aptamer_sequences.fasta blat_flybrain_natural_vs_generated_aptamers_output.psl
Loaded 52332500 letters in 104665 sequences
Searched 1568512 bases in 12254 sequences


In [43]:
df = parse_psl(out_psl)

In [44]:
top_hits = summarize_blat_results(df)


Top-hit identity statistics:
count    112.000000
mean       0.978894
std        0.025257
min        0.916667
25%        0.968367
50%        0.990385
75%        1.000000
max        1.000000
Name: percent_identity, dtype: float64

Perfect duplicates (identity=1.0 and full coverage): 0 / 112


In [45]:
plot_alignment_stats(top_hits)